In [1]:
import pandas as pd
import numpy as np
import collections

## 統計各牙齒種類的準確率

In [2]:
molar    = [ 1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
premolar = [ 4, 5, 12, 13, 20, 21, 28, 29 ]
canine   = [ 6, 11, 22, 27  ]
incisors = [ 7, 8, 9, 10, 23, 24, 25, 26 ]


tooth_type_dict = { tooth_type: tooth_list for tooth_type, tooth_list in 
                    zip([ "molar", "premolar", "canine", "incisors" ],\
                        [molar, premolar, canine, incisors])  
                  }

def accuracy(data, title, filter_list):
        total_data = data[ (data["tooth_num"].isin(filter_list)) ]
        match_data = data[ (data["tooth_num"].isin(filter_list)) & (data["Match"] == 1) ]
        
        
        if len(total_data) == 0 : accuracy = 0
        else : accuracy = round(len(match_data) / len(total_data) * 100, 2)
        
        return len(total_data), len(match_data), accuracy
#         print(f"Accuracy for {title} : {accuracy} %")

fold_accuracy_table, data_volume_table, match_table = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
accuracy_dict, data_volume_dict, match_dict = collections.OrderedDict(), collections.OrderedDict(), collections.OrderedDict()

for fold_num in range(1, 6):
        path = f"Results/Regular_Inception_Multitask_binary_Fold_520201102-173023/Fold_{fold_num}/table/test_predict.csv"
        data = pd.read_csv(path)
        match = [ 1 if gt == predict else 0 for gt, predict in zip(data.Class, data.Predict) ]
        data["Match"] = match
        
        item, item_volume, item_match = {}, {}, {}
        for tooth_type, tooth_list in tooth_type_dict.items():
                total, match, acc = accuracy(data, tooth_type, tooth_list)
                item_volume[tooth_type] = total
                item_match[tooth_type]  = match
                item[tooth_type] = acc
        
        item[f"Total_Acc"] = round( len( data[ data["Match"] == 1 ] ) / len( data ) * 100, 2)      
        
        
        accuracy_dict[ f"Fold_{fold_num}" ]     = item
        data_volume_dict[ f"Fold_{fold_num}" ]  = item_volume
        match_dict[ f"Fold_{fold_num}" ]        = item_match
 

results = fold_accuracy_table.from_dict(accuracy_dict).T
accuracy_dict["Tooth_Acc"] = round(results.mean(axis=0), 2)
results = fold_accuracy_table.from_dict(accuracy_dict).T

results

,molar,premolar,canine,incisors,Total_Acc
Fold_1,85.94,87.67,91.50,91.61,88.63
Fold_2,81.89,87.60,84.09,86.87,85.11
Fold_3,85.82,85.26,86.61,88.30,86.49
Fold_4,81.40,86.62,86.45,85.99,84.62
Fold_5,84.84,84.43,92.59,88.98,86.76
Tooth_Acc,83.98,86.32,88.25,88.35,86.32


In [3]:
data_volume_results = data_volume_table.from_dict(data_volume_dict).T
data_volume_results

,molar,premolar,canine,incisors
Fold_1,11471,9007,3375,10067
Fold_2,11477,9011,3376,10056
Fold_3,11076,9245,3479,10120
Fold_4,11673,8892,3298,10057
Fold_5,11078,9244,3484,10114


In [4]:
match_results = match_table.from_dict(match_dict).T
match_results

,molar,premolar,canine,incisors
Fold_1,9858,7896,3088,9222
Fold_2,9399,7894,2839,8736
Fold_3,9505,7882,3013,8936
Fold_4,9502,7702,2851,8648
Fold_5,9399,7805,3226,8999


## 統計原來各牙齒種類的資料個數

In [5]:
all_path = f"tmp.csv"
all_data = pd.read_csv(all_path)

for tooth_type, tooth_list in tooth_type_dict.items():
        total_data = all_data[ (all_data["tooth_num"].isin(tooth_list)) ]
        print( f"Original # of tooth type - {tooth_type} : {len(total_data) // 80 } " )
        
        for i in range(3):
                total_data = all_data[ (all_data["tooth_num"].isin(tooth_list)) & (all_data["Class"] == i) ]
                print(f"        # of class {i} : { len(total_data) // 80}")
        

Original # of tooth type - molar : 0 
        # of class 0 : 0
        # of class 1 : 0
        # of class 2 : 0
Original # of tooth type - premolar : 2399 
        # of class 0 : 2043
        # of class 1 : 259
        # of class 2 : 97
Original # of tooth type - canine : 0 
        # of class 0 : 0
        # of class 1 : 0
        # of class 2 : 0
Original # of tooth type - incisors : 0 
        # of class 0 : 0
        # of class 1 : 0
        # of class 2 : 0


In [6]:
def accuracy(data, title, filter_list):
        total_data = data[ (data["tooth_num"].isin(filter_list)) ]
        match_data = data[ (data["tooth_num"].isin(filter_list)) & (data["Match"] == 1) ]
        accuracy = round(len(match_data) / len(total_data) * 100, 2)
        return accuracy